In [1]:
import sys
import os
import cv2
os.environ['CUDA_VISIBLE_DEVICES'] = '/gpu:0'
sys.path.append('/backup1/xymmodel/HandPose')
sys.path.append('/backup1/xymmodel/yolo')
from detect_cell_phone import EfficientDet
from detect_finger import detect_finger
from yolo import YOLO
from PIL import Image
import numpy as np
yolo = YOLO(model_path="/backup1/xymmodel/yolo/model_data/B5_243.h5")

/home/user/anaconda3/envs/tensorflow-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/user/anaconda3/envs/tensorflow-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/user/anaconda3/envs/tensorflow-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/user/anaconda3/envs/te

Instructions for updating:
Colocations handled automatically by placer.
/backup1/xymmodel/yolo/model_data/B5_243.h5 model, anchors, and classes loaded.


In [2]:
def inside(point, box):
        return box[0] < point[0] < box[2] and box[1] < point[1] < box[3]

In [25]:
def get_larger_box(xy, scale, w, h):
    """
    Args:
        xy: bounding boxes in boundary coordinates, a tensor of size (n_boxes, 4)
        scale: the scale of box
    Returns: 
        bounding boxes in center-size coordinates, a tensor of size (n_boxes, 4)
    """
    xy = np.reshape(xy, (1, 4))
    cx_cy = np.concatenate(((xy[:, 2:] + xy[:, :2]) / 2,  # c_x, c_y
                      (xy[:, 2:] - xy[:, :2]) * scale), axis = 1) # w, h
    # 将直接放大scale倍改为仅放大小边长度
    # min的值是较短的边乘以scale
    min = np.array([cx_cy[:, 3][i] if(cx_cy[:, 3][i] < cx_cy[:, 2][i]) else cx_cy[:, 2][i] for i in range(len(cx_cy[:, 3]))], dtype=np.int16)
    left_y = cx_cy[:, 1] - min * 0.5
    left_x = cx_cy[:, 0] - 0.5 * min
    right_y = cx_cy[:, 1] + 0.5 * min
    right_x = cx_cy[:, 0] + 0.5 * min
    left_y = np.array([item if item > 0 else 0 for item in left_y], dtype=np.int16)
    left_x = np.array([item if item > 0 else 0 for item in left_x], dtype=np.int16)
    right_y = np.array([item if item <= h else h for item in right_y], dtype=np.int16)
    right_x = np.array([item if item <= w else w for item in right_x], dtype=np.int16)
    return np.stack((left_x, left_y, right_x, right_y), axis = 1) # w, h

def find_start(video_path):
    video = cv2.VideoCapture(video_path)
    videoName = os.path.splitext(os.path.split(video_path)[1])[0]
    count = 0
    if not video.isOpened():
        print("Couldn't open the video...")
    while True:
        count += 1
        ret, frame = video.read()
        if count < 100 or count % 5:
            continue
        if not ret:
            break

        # 第一步：检测手机，检测不到手机则跳过当前帧。
        PILFrame = Image.fromarray(frame)
        _, phone = yolo.detect_image(PILFrame)
        phone = phone[0]
        if phone is None or len(phone) < 4:
            print(f'第{count}帧，检测不到手机...')
            continue

        # 第二步：检测指尖。这步里会对手机区域放大后进行裁剪，只检测裁剪区域内是否存在指尖，检测不到则跳过当前帧。
        big_box = get_larger_box(phone, 4, frame.shape[1], frame.shape[0])[0]
        mid_box = get_larger_box(phone, 1.9, frame.shape[1], frame.shape[0])[0]
        threshold = 0.3
        finger = detect_finger(frame, big_box, threshold, only_index_finger=False)
        print(finger)
        if not finger:
            print(f'第{count}帧，检测到手机但检测不到手指...')
            continue
        # 第三步：如果检测到了指尖，且该指尖区域在两倍到四倍手机尺寸范围内，才认为是右手指尖。
        else:
            if len(finger) > 22:
                print(f"开始帧为第{count}帧...")
                break
            else:
                print(f"第{count}帧右手未进入")
    return {
        'frame':frame,
        'count':count,
        'phone':phone,
        'finger':finger,
        'phone_4_times':big_box,
        'mid_box':mid_box
    }


In [26]:
video = "SSSM-B5-042"
videoDir = "/home/user/zy/attack-on-pattern-pin/data/batch_5/"
outputDir = "/home/user/zy/attack-on-pattern-pin/results/batch_5/"
videoNames = os.listdir(videoDir)
videoNames.sort(key=lambda x: x[-7:-4])
for videoName in videoNames:
    name, vType = os.path.splitext(videoName)
    if (vType != ".mp4"):
        continue
    if (name != video):
        continue
    if (int(name[-2:]) <= 34): continue
    print(name)
    ret = find_start(videoDir + videoName)
    if not ret['finger'] or len(ret['finger']) < 18:
        continue
    if not os.path.exists(os.path.join(outputDir, name)):
        os.mkdir(os.path.join(outputDir, name))

    cv2.rectangle(ret['frame'],(ret['phone_4_times'][0], ret['phone_4_times'][1]),(ret['phone_4_times'][2], ret['phone_4_times'][3]),(0,255,0),3)
    cv2.imwrite(os.path.join(outputDir, name, "start.jpg"), ret['frame'])
    with open(os.path.join(outputDir, name, "start2.txt"), "w") as f:
        f.write(str(ret['count']))
#     break
    

SSSM-B5-042
第100帧，检测不到手机...
第105帧，检测不到手机...
第110帧，检测不到手机...
第115帧，检测不到手机...
第120帧，检测不到手机...
第125帧，检测不到手机...
第130帧，检测不到手机...
第135帧，检测不到手机...
[[0.4306161105632782, [1363, 426]], [0.502587080001831, [1364, 432]], [0.5527404546737671, [1369, 433]], [0.6296466588973999, [1382, 433]], [0.5037797689437866, [1390, 433]], [0.4206670820713043, [1409, 459]], [0.34040942788124084, [1396, 498]], [0.4596034288406372, [1383, 505]], [0.4580317735671997, [1422, 471]], [0.5859948396682739, [1409, 498]], [0.6062257289886475, [1396, 504]], [0.6262022852897644, [1382, 511]], [0.5170938968658447, [1428, 472]], [0.593989372253418, [1415, 492]], [0.5254083275794983, [1396, 504]], [0.46665894985198975, [1389, 504]], [0.5042676329612732, [1428, 472]], [0.3488420844078064, [1416, 491]], [0.36708080768585205, [1402, 498]]]
第140帧右手未进入
[[0.5336354970932007, [1391, 457]], [0.3695632219314575, [1391, 457]], [0.31711170077323914, [1411, 464]], [0.43207812309265137, [1429, 471]], [0.44834068417549133, [1444, 494]], [0.

In [12]:
import cv2
print(ret['finger'], ret['phone_4_times'])
cv2.rectangle(ret['frame'],(ret['phone'][0], ret['phone'][1]), (ret['phone'][2], ret['phone'][3]),(0,255,0),3)
cv2.rectangle(ret['frame'],(ret['mid_box'][0], ret['mid_box'][1]),(ret['mid_box'][2], ret['mid_box'][3]),(0,255,0),3)
cv2.rectangle(ret['frame'],(ret['phone_4_times'][0], ret['phone_4_times'][1]),(ret['phone_4_times'][2], ret['phone_4_times'][3]),(0,255,0),3)
cv2.circle(ret['frame'], tuple(ret['finger'][0][1]), 3, (0,0,255), -1)
# cv2.circle(ret['frame'], tuple(ret['finger'][1][1]), 3, (0,0,255), -1)
cv2.imwrite('/home/user/zy/attack-on-pattern-pin/data/temp2.jpg', ret['frame'])

[[0.6815094947814941, [1074, 356]], [0.511264979839325, [1082, 350]], [0.5740032196044922, [1071, 373]], [0.6206931471824646, [1082, 359]], [0.6904919743537903, [1088, 351]], [0.7164154052734375, [1096, 345]], [0.47652554512023926, [1076, 379]], [0.4518216550350189, [1085, 368]], [0.6963211894035339, [1091, 364]], [0.5022012591362, [1094, 362]], [0.5458832383155823, [1082, 387]], [0.6494020223617554, [1088, 379]], [0.7412198185920715, [1096, 373]], [0.661288857460022, [1099, 371]], [0.510644257068634, [1090, 394]], [0.5791364908218384, [1096, 388]], [0.6191408038139343, [1100, 382]], [0.6142181158065796, [1103, 379]]] [1044  272 1176  404]


True